**Table of contents**<a id='toc0_'></a>    
1. [迭代器(iterator)](#toc1_)    
1.1. [目的](#toc1_1_)    
1.2. [什么是"迭代"](#toc1_2_)    
1.3. [可迭代对象](#toc1_3_)    
1.4. [迭代器](#toc1_4_)    
1.4.1. [可迭代对象的本质](#toc1_4_1_)    
1.4.2. [获取可迭代对象的迭代器](#toc1_4_2_)    
1.4.3. [获取迭代器的数据](#toc1_4_3_)    
1.4.4. [StopIteration异常](#toc1_4_4_)    
1.5. [自定义"迭代器"](#toc1_5_)    
1.5.1. [__iter__方法](#toc1_5_1_)    
1.5.2. [__next__方法](#toc1_5_2_)    
1.5.3. [自定义”迭代器“案例](#toc1_5_3_)    
1.6. [for...in...循环的本质](#toc1_6_)    
1.7. [并不是只有for循环能接收可迭代对象](#toc1_7_)    
1.8. [总结](#toc1_8_)    
1.9. [作业](#toc1_9_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=true
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# 1. <a id='toc1_'></a>[迭代器(iterator)](#toc0_)

## 1.1. <a id='toc1_1_'></a>[目的](#toc0_)
如果开发中有如下需求，该怎样解决呢？

In [1]:
class StuSystem(object):
    """
    学生管理系统
    """
    def __init__(self):
        self.stus = []

    def add(self):
        """
        添加一个新的学生
        :return:
        """
        name = input("请输入新学生的姓名:")
        tel = input("请输入新学生的手机号:")
        address = input("请输入新学生的住址:")

        new_stu = {"name": name, "tel": tel, "address": address}
        self.stus.append(new_stu)


# 创建管理系统对象
stu_sys = StuSystem()

# 添加3个学生信息到系统中
stu_sys.add()
stu_sys.add()
stu_sys.add()

# 问题1：怎样才能实现用for循环遍历系统中所有的学生信息呢？下面的方式能实现吗？
for temp in stu_sys:
    print(temp)

# 问题2：如果需要一个列表，这个列表 样子例如 [("张三", "10086"), ("李四", "10010")]
# stu_list = [ ...列表推导式...]
# 这个列表推导式该怎样写才能实现呢？


TypeError: 'StuSystem' object is not iterable

在实际开发工作中，经常需要快速的将对象转化问其他的不同的数据类型，此时如果能快速的遍历出自定义的对象，这样大大减少代码的冗余，而且可读性会更优美

问题是，怎样实现呢？

今天我们要学习的知识只有1个，那就是”迭代器“


## 1.2. <a id='toc1_2_'></a>[什么是"迭代"](#toc0_)
迭代是访问集合元素的一种方式

例如

In [2]:
nums = [11, 22, 33]

# 可以通过for循环将nums列表中的每个数据依次获取
for num in nums:
    print(num)


name = "teacher"

for temp in name:
    print(temp)


11
22
33
t
e
a
c
h
e
r


我们已经知道可以对list、tuple、str等类型的数据使用for...in...的循环语法从其中依次拿到数据进行使用，我们把这样的过程称为遍历，也叫迭代

## 1.3. <a id='toc1_3_'></a>[可迭代对象](#toc0_)
是否所有的数据类型都可以放到for...in...的语句中，然后让for...in...每次从中取出一条数据供我们使用呢？

看如下示例：

In [3]:
nums = [11, 22, 33]

# 可以通过for循环将nums列表中的每个数据依次获取
for num in nums:
    print(num)
    
weight = 180

for temp in weight:
    print(temp)

11
22
33


TypeError: 'int' object is not iterable


可以发现，并不是所有的类型都可以通过for...in...的方式进行遍历

我们可以通俗的认为：

 > 只要是可以通过for...in…的形式进行遍历的，那么这个数据类型就是可以迭代的

例如，下面的是可以迭代的数据类型
- 列表
- 元组
- 字典
- 字符串

而下面的则不是可以迭代的数据类型
- 整型
- 浮点型

那是否可以通过某种方式能够测量出一个数据类型到底是否是可以迭代呢？看下面的知识

In [4]:
from collections.abc import Iterable

print(f'列表是否是可迭代的——>: {isinstance([], Iterable)}')
print(f'元组是否是可迭代的——>: {isinstance((), Iterable)}')
print(f"字典是否是可迭代的——>: {isinstance({'name': 'jason', 'age': 20}, Iterable)}")
print(f"集合是否是可迭代的——>: {isinstance({'name', 'age',1}, Iterable)}")
print(f'字符串是否是可迭代的——>: {isinstance("abc", Iterable)}')

print(f'整数是否是可迭代的——>: {isinstance(100, Iterable)}')
print(f'浮点数是否是可迭代的——>: {isinstance(1.1, Iterable)}')


列表是否是可迭代的——>: True
元组是否是可迭代的——>: True
字典是否是可迭代的——>: True
集合是否是可迭代的——>: True
字符串是否是可迭代的——>: True
整数是否是可迭代的——>: False
浮点数是否是可迭代的——>: False


**只要是通过isinstance来判断出是Iterable类的实例，即isinstance的结果是True那么就表示，这个数据类型是可以迭代的数据类型**

## 1.4. <a id='toc1_4_'></a>[迭代器](#toc0_)
迭代器是一个可以记住遍历的位置的对象。迭代器对象从第一个元素开始访问，直到所有的元素被访问完结束。迭代器只能往前不会后退。

### 1.4.1. <a id='toc1_4_1_'></a>[可迭代对象的本质](#toc0_)
分析 可迭代对象 进行迭代的过程，发现每迭代一次（即在for...in...中每循环一次）都会返回对象中的下一条数据，一直向后读取数据直到迭代了所有数据后结束。

那么，在这个过程中就应该有一个“人”去记录每次访问到了第几条数据，以便每次迭代都可以返回下一条数据。我们把这个能帮助我们进行数据迭代的“人”称为迭代器(Iterator)

可迭代对象的本质就是可以向我们提供一个这样的中间“人”，即迭代器帮助我们对其进行迭代遍历使用。

list、tuple等都是可迭代对象，我们可以通过iter()函数获取这些可迭代对象的迭代器。然后我们可以对获取到的迭代器不断使用next()函数来获取下一条数据。

### 1.4.2. <a id='toc1_4_2_'></a>[获取可迭代对象的迭代器](#toc0_)
测试代码如下:

In [5]:
from collections.abc import Iterator

nums = [11, 22, 33, 44]

print(type(nums))

nums_iter = iter(nums)

print(type(nums_iter))

print("nums", isinstance(nums, Iterator))
print("nums_iter", isinstance(nums_iter, Iterator))

<class 'list'>
<class 'list_iterator'>
nums False
nums_iter True


### 1.4.3. <a id='toc1_4_3_'></a>[获取迭代器的数据](#toc0_)
上面提到，通过iter()能够得到一个可迭代对象的 迭代器，可以通过next()函数多次提取迭代器中的数据，下面我们就测试下

测试代码如下：

In [6]:
from collections.abc import Iterator


nums = [11, 22, 33, 44]
nums_iter = iter(nums)

print("nums", isinstance(nums, Iterator))
print("nums_iter", isinstance(nums_iter, Iterator))


num1 = next(nums_iter)
print(num1)

num2 = next(nums_iter)
print(num2)

num3 = next(nums_iter)
print(num3)

num4 = next(nums_iter)
print(num4)


nums False
nums_iter True
11
22
33
44


### 1.4.4. <a id='toc1_4_4_'></a>[StopIteration异常](#toc0_)
如果将上面的代码，多写一次的next()会怎样呢？看如下测试代码：

In [7]:
from collections.abc import Iterator


nums = [11, 22, 33, 44]
nums_iter = iter(nums)

print("nums", isinstance(nums, Iterator))
print("nums_iter", isinstance(nums_iter, Iterator))


num1 = next(nums_iter)
print(num1)

num2 = next(nums_iter)
print(num2)

num3 = next(nums_iter)
print(num3)

num4 = next(nums_iter)
print(num4)

num5 = next(nums_iter)  # 这里会产生异常
print(num5)


nums False
nums_iter True
11
22
33
44


StopIteration: 

可以看到23行，即第5次调用next()时，产生了异常。why？？？？？？

因为列表nums中只有4个数据，也就是说可以调用4次next是完全合理的，但是如果，调用的次数多了肯定是不合理，都没有5个数据，怎么可以能取5次呢！显然是不对的

此时估计想明白了，为什么会产生异常，其实就是一种告知迭代结束的标志而已

添加try...except...即可解决刚刚遇到的问题

In [8]:
from collections.abc import Iterator


nums = [11, 22, 33, 44]
nums_iter = iter(nums)

print("nums", isinstance(nums, Iterator))
print("nums_iter", isinstance(nums_iter, Iterator))


num1 = next(nums_iter)
print(num1)

num2 = next(nums_iter)
print(num2)

num3 = next(nums_iter)
print(num3)

num4 = next(nums_iter)
print(num4)

try:
    num5 = next(nums_iter)  # 这里会产生异常
    print(num5)
except StopIteration as e:
    print(e)


nums False
nums_iter True
11
22
33
44



## 1.5. <a id='toc1_5_'></a>[自定义"迭代器"](#toc0_)
大家是否还记得 在刚开学习今天的知识时，我们引入了一个学生管理系统的问题，该怎样实现呢

我们下面来谈谈

- __iter__方法
- __next__方法

### 1.5.1. <a id='toc1_5_1_'></a>[__iter__方法](#toc0_)
上面提到iter()方法必须是对”可迭代“对象 才能 提取到 ”迭代器“对象，但是怎样保证自定义的对象是”可迭代“对象呢？

答：

>只要在类中，定义__iter__方法，那么这个类创建出来的对象一定是可迭代对象

通俗的说：一个具备了__iter__方法的对象，就是一个可迭代对象

测试代码如下:（无__iter__方法）

In [1]:
from collections.abc import Iterable


class MyList(object):
    def __init__(self):
        self.container = []

    def add(self, item):
        self.container.append(item)

        
mylist = MyList()
mylist.add(11)
mylist.add(22)
mylist.add(33)

print("mylist是否是可以迭代对象", isinstance(mylist, Iterable))

for temp in mylist:
    print(temp)


mylist是否是可以迭代对象 False


TypeError: 'MyList' object is not iterable

测试代码2：(有__iter__方法)

In [2]:
from collections.abc import Iterable


class MyList(object):
    def __init__(self):
        self.container = []

    def add(self, item):
        self.container.append(item)

    def __iter__(self):
        pass

mylist = MyList()
mylist.add(11)
mylist.add(22)
mylist.add(33)

print("mylist是否是可以迭代对象", isinstance(mylist, Iterable))

for temp in mylist:
    print(temp)


mylist是否是可以迭代对象 True


TypeError: iter() returned non-iterator of type 'NoneType'

能够看出，一个类，只要有__iter__方法，那么这个类创建出来的对象就是可以迭代对象

其实，当我们调用iter()函数提取一个可迭代对象的 迭代器时，实际上会自动调用这个对象的__iter__方法，并且这个方法返回迭代器

### 1.5.2. <a id='toc1_5_2_'></a>[__next__方法](#toc0_)
通过上面的分析，我们已经知道，迭代器是用来帮助我们记录每次迭代访问到的位置，当我们对迭代器使用next()函数的时候，迭代器会向我们返回它所记录位置的下一个位置的数据。

实际上，在使用next()函数的时候，调用的就是迭代器对象的__next__方法（Python3中是对象的__next__方法，Python2中是对象的next()方法）。

所以，我们要想构造一个迭代器，就要实现它的__next__方法。

但这还不够，python要求迭代器本身也是可迭代的，所以我们还要为迭代器实现__iter__方法，而__iter__方法要返回一个迭代器，迭代器自身正是一个迭代器，所以迭代器的__iter__方法返回自身即可。

一个实现了__iter__方法和__next__方法的对象，就是迭代器

如何判断一个对象是否是迭代器

可以使用 isinstance() 判断一个对象是否是 Iterator 对象：

In [3]:
from collections.abc import Iterator

print(isinstance([],Iterator))

print(isinstance(iter([]),Iterator))

print(isinstance(iter("abc"), Iterator))

False
True
True


自定迭代器

In [4]:
from collections.abc import Iterable
from collections.abc import Iterator


class MyList(object):
    """自定义的一个可迭代对象"""
    def __init__(self):
        self.items = []

    def add(self, val):
        self.items.append(val)

    def __iter__(self):
        return MyIterator()


class MyIterator(object):
    """自定义的供上面可迭代对象使用的一个迭代器"""
    def __init__(self):
        pass

    def __next__(self):
        pass

    def __iter__(self):
        pass


mylist = MyList()
mylist_iter = iter(mylist)

print("mylist是否是可以迭代对象", isinstance(mylist, Iterable))
print("mylist是否是迭代器", isinstance(mylist, Iterator))

print("mylist_iter是否是可以迭代对象", isinstance(mylist_iter, Iterable))
print("mylist_iter是否是迭代器", isinstance(mylist_iter, Iterator))



mylist是否是可以迭代对象 True
mylist是否是迭代器 False
mylist_iter是否是可以迭代对象 True
mylist_iter是否是迭代器 True


### 1.5.3. <a id='toc1_5_3_'></a>[自定义”迭代器“案例](#toc0_)

In [5]:
class MyList(object):
    """自定义的一个可迭代对象"""
    def __init__(self):
        self.items = []

    def add(self, val):
        self.items.append(val)

    def __iter__(self):
        return MyIterator(self)


class MyIterator(object):
    """自定义的供上面可迭代对象使用的一个迭代器"""
    def __init__(self, mylist):
        self.mylist = mylist
        # current用来记录当前访问到的位置
        self.current = 0

    def __next__(self):
        if self.current >= len(self.mylist.items):
            raise StopIteration
        item = self.mylist.items[self.current]
        self.current += 1
        return item

    def __iter__(self):
        return self


if __name__ == '__main__':
    mylist = MyList()
    mylist.add(1)
    mylist.add(2)
    mylist.add(3)
    mylist.add(4)
    mylist.add(5)
    for num in mylist:
        print(num)


1
2
3
4
5


可迭代对象通过__iter__方法向我们返回一个迭代器，我们在迭代一个可迭代对象的时候，实际上就是先获取该对象提供的一个迭代器，然后通过这个迭代器来依次获取对象中的每一个数据.

## 1.6. <a id='toc1_6_'></a>[for...in...循环的本质](#toc0_)
1. 先调用iter()函数，它会自动调用可迭代对象中的__iter__方法，此方法返回这个可迭代对象的 迭代器对象
2. 对获取到的迭代器不断调用next()函数，它会自动调用迭代器中的__next__方法来获取下一个值
3. 当遇到StopIteration异常后循环结束

## 1.7. <a id='toc1_7_'></a>[并不是只有for循环能接收可迭代对象](#toc0_)
除了for循环能接收可迭代对象，list、tuple等也能接收。

测试代码如下：

In [6]:
class MyList(object):
    """自定义的一个可迭代对象"""
    def __init__(self):
        self.items = []

    def add(self, val):
        self.items.append(val)

    def __iter__(self):
        return MyIterator(self)


class MyIterator(object):
    """自定义的供上面可迭代对象使用的一个迭代器"""
    def __init__(self, mylist):
        self.mylist = mylist
        # current用来记录当前访问到的位置
        self.current = 0

    def __next__(self):
        if self.current >= len(self.mylist.items):
            self.current= 0
            raise StopIteration
        item = self.mylist.items[self.current]
        self.current += 1
        return item

    def __iter__(self):
        return self


if __name__ == '__main__':
    mylist = MyList()
    mylist.add(1)
    mylist.add(2)
    mylist.add(3)
    mylist.add(4)
    mylist.add(5)

    print(list(mylist))

[1, 2, 3, 4, 5]


## 1.8. <a id='toc1_8_'></a>[总结](#toc0_)
- 凡是可作用于for 循环的对象都是 Iterable 类型；
- 凡是可作用于 next() 函数的对象都是 Iterator 类型
- 集合数据类型如 list 、dict、str等是 Iterable 但不是Iterator，不过可以通过 iter() 函数获得一个 Iterator 对象

## 1.9. <a id='toc1_9_'></a>[作业](#toc0_)
既然已经学习过了迭代器，那么今天刚开始的知识点也就自然有了答案

问题1的参考代码如下：

In [13]:
class StuSystem(object):
    """
    学生管理系统
    """
    def __init__(self):
        self.stus = []
        self.current =0

    def add(self):
        """
        添加一个新的学生
        :return:
        """
        name = input("请输入新学生的姓名:")
        tel = input("请输入新学生的手机号:")
        address = input("请输入新学生的住址:")

        new_stu = {"name": name, "tel": tel, "address": address}
        self.stus.append(new_stu)
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.current >= len(self.stus):
            self.current=0 #* 置为0
            raise StopIteration
        item=self.stus[self.current]
        self.current+=1
        return item


# 创建管理系统对象
stu_sys = StuSystem()

# 添加3个学生信息到系统中
stu_sys.add()
stu_sys.add()
stu_sys.add()

# 问题1：怎样才能实现用for循环遍历系统中所有的学生信息呢？下面的方式能实现吗？
for temp in stu_sys:
    print(temp)
    
for temp in stu_sys:
    print(temp)

# 问题2：如果需要一个列表，这个列表 样子例如 [("张三", "10086"), ("李四", "10010")]
# stu_list = [ ...列表推导式...]
# 这个列表推导式该怎样写才能实现呢？


{'name': '1', 'tel': '1', 'address': '1'}
{'name': '2', 'tel': '2', 'address': '2'}
{'name': '3', 'tel': '3', 'address': '3'}
{'name': '1', 'tel': '1', 'address': '1'}
{'name': '2', 'tel': '2', 'address': '2'}
{'name': '3', 'tel': '3', 'address': '3'}


问题2的参数代码如下：

In [14]:
class StuSystem(object):
    """
    学生管理系统
    """
    def __init__(self):
        self.stus = []
        self.current =0

    def add(self):
        """
        添加一个新的学生
        :return:
        """
        name = input("请输入新学生的姓名:")
        tel = input("请输入新学生的手机号:")
        address = input("请输入新学生的住址:")

        new_stu = {"name": name, "tel": tel, "address": address}
        self.stus.append(new_stu)
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.current >= len(self.stus):
            self.current=0 #* 置为0
            raise StopIteration
        item=self.stus[self.current]
        self.current+=1
        return item


# 创建管理系统对象
stu_sys = StuSystem()

# 添加3个学生信息到系统中
stu_sys.add()
stu_sys.add()
stu_sys.add()

# 问题1：怎样才能实现用for循环遍历系统中所有的学生信息呢？下面的方式能实现吗？
for temp in stu_sys:
    print(temp)

# 问题2：如果需要一个列表，这个列表 样子例如 [("张三", "10086"), ("李四", "10010")]
# stu_list = [ ...列表推导式...]
# 这个列表推导式该怎样写才能实现呢？
stu_list =[ temp for temp in stu_sys ]
print(stu_list)


{'name': '1', 'tel': '1', 'address': '1'}
{'name': '2', 'tel': '2', 'address': '2'}
{'name': '3', 'tel': '3', 'address': '3'}
[{'name': '1', 'tel': '1', 'address': '1'}, {'name': '2', 'tel': '2', 'address': '2'}, {'name': '3', 'tel': '3', 'address': '3'}]
